In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [5]:
Main_directory = 'C:/Users/Iwan/Documents/GitHub/Iwan-and-George-Year-4-Project/Exoplanets'
All_stars_directory = f'{Main_directory}/All Stars Data.csv'

All_stars_data = pd.read_csv(All_stars_directory)

def Data_check(Data, headers=False):
    headers = Data.columns if type(headers) is bool else headers 
    bool_array = [pd.notna(Data[col_name]) for col_name in headers]
    check = np.flipud(np.rot90(bool_array))
    return check

def Find_least_error(Data, Checked_data, headers, error_headers, pm_seperation=True):
    least_error_array = []
    for row, boolean in enumerate(Checked_data):
        previous_error = np.inf
        for dataset, error_header in enumerate(error_headers):
            if boolean[dataset]:
                if pm_seperation:
                    current_error = Data[error_header[0]][row] + Data[error_header[1]][row]
                else:
                    current_error = 2*Data[error_header][row]
                if current_error < previous_error:
                    previous_error = current_error
                    lowest_error = headers[dataset]
        least_error_array.append(lowest_error)
    return np.array(least_error_array)

In [6]:
Teff_headers = ['Teff (Gaia)', 'Teff (Exo)', 'Teff (Fulton)', 'Teff (Van Eylen)']
Checked_data = Data_check(All_stars_data, Teff_headers)

Teff_error_headers = [['Teff pe (Gaia)', 'Teff me (Gaia)'], 
                      ['Teff pe (Exo)', 'Teff me (Exo)'], 
                      ['Teff pe (Fulton)', 'Teff me (Fulton)'],
                      ['Teff pe (Van Eylen)', 'Teff me (Van Eylen)']]
            
Best_teff_sources = Find_least_error(All_stars_data, Checked_data, Teff_headers, Teff_error_headers)